# Question 1: Manual Value Iteration

- Initialize Value Function: $v_0(s_1) = 10, v_0(s_2)=1, v_0(s_3)$.
- Iteration 1: 

$$\begin{aligned}
q_1(s_1, a_1) &= 8 +0.2*10+0.6*1+0=10.6\\
q_1(s_1, a_2) &= 10 +0.1*10+0.2*1+0=11.2\\
v_1(s_1) &= max(10.6, 11.2) = 11.2\\
\pi_1(s_1) &= a_2
\end{aligned}$$

$$\begin{aligned}
q_1(s_2, a_1) &= 1 +0.3*10+0.3*1+0=4.3\\
q_1(s_2, a_2) &= -1 +0.5*10+0.3*1+0=4.3\\
v_1(s_2) &= max(4.3, 4.3) = 4.3\\
\pi_1(s_2) &= a_1
\end{aligned}$$

- Iteration 2:

$$\begin{aligned}
q_2(s_1, a_1) &= 8 +0.2*11.2+0.6*4.3+0=12.82\\
q_2(s_1, a_2) &= 10 +0.1*11.2+0.2*4.3+0=11.98\\
v_2(s_1) &= max(12.82, 11.98) = 12.82\\
\pi_2(s_1) &= a_1
\end{aligned}$$

$$\begin{aligned}
q_2(s_2, a_1) &= 1 +0.3*11.2+0.3*4.3+0=5.65\\
q_2(s_2, a_2) &= -1 +0.5*11.2+0.3*4.3+0=5.89\\
v_2(s_2) &= max(5.65,5.89) =5.89\\
\pi_2(s_2) &= a_2
\end{aligned}$$

- Why only needs k=2:
$$\begin{aligned}
q_k(s_1, a_1) - q_k(s_1, a_2) &= 8-10 + (0.2-0.1)v_{k-1}(s_1)+ (0.6-0.2)v_{k-1}(s_2)\\
&= -2 + 0.1v_{k-1}(s_1)+ 0.4 v_{k-1}(s_2) ~ \text{for all } k\geq 1\\
q_k(s_2, a_1) - q_k(s_2, a_2) &= 1-(-1) + (0.3-0.5)v_{k-1}(s_1)+ (0.3-0.3)v_{k-1}(s_2)\\
&= 2 -0.2v_{k-1}(s_1) ~ \text{for all } k\geq 1
\end{aligned}$$

Since $v_k(s_1) \geq 12.82$ and $v_k(s_2) \geq 5.89$ for all $k \geq 3$, we have
$$\begin{aligned}
q_k(s_1, a_1) - q_k(s_1, a_2) \geq -2 + 0.1*12.82+ 0.4 *5.89 = 1.638 >0 ~ \text{for all } k\geq 3\\
q_k(s_2, a_1) - q_k(s_2, a_2) \leq 2 -0.2*12.82 = -0.56 <0  ~ \text{for all } k\geq 3
\end{aligned}$$

Hence $q_k(s_1, a_1) >q_k(s_1, a_2)$ and  $q_k(s_2, a_2) >q_k(s_2, a_1)$ for all $k\geq 3$. Therefore we have $\pi_k(s_1) = a_1$ and $\pi_k(s_2) = a_2$ for all $k\geq3$.

# Question 2: Frog Croaking revisited

(refer back to Q3 on Assignment 3)

Task: 
- Compute Optimal Value Function and Optimal Policy Using Policy Iteration Algorithm & Value Iteration Algorithm;  - Analyze computational efficiency in terms of speed of convergence to the optimal value function. 
- How do those 2 iteration methods compare with brute force method of evaluating MDP for $2^{n-1}$ determimistic policy in previous assignment?
- Plot graphs of convergence for different values of n.

In [19]:
import sys
sys.path
sys.path.append('/Users/mulingsi/Desktop/MSE346/RL-book/')
from dataclasses import dataclass
from typing import Tuple, Dict
from rl.markov_decision_process import FiniteMarkovDecisionProcess
from rl.markov_decision_process import FinitePolicy, StateActionMapping
from rl.markov_process import FiniteMarkovProcess, FiniteMarkovRewardProcess
from rl.distribution import Categorical, Constant
from scipy.stats import poisson
import pandas as pd

In [5]:
@dataclass(frozen=True)
class LilypadState:
    lilypad: int
        
LilypadSoundMapping = StateActionMapping[LilypadState, str]

class FrogEscapeMDP(FiniteMarkovDecisionProcess[LilypadState, str]):
    
    def __init__(self, num_lilypads:int):
        self.num_lilypads = num_lilypads 
        super().__init__(self.get_action_transition_reward_map())

    def get_action_transition_reward_map(self) -> LilypadSoundMapping:
        d: Dict[LilypadState, Dict[str, Categorical[Tuple[LilypadState,float]]]] = {}
            
        d[0] = None
        d[self.num_lilypads] = None
        
        for lilypad in range(1,self.num_lilypads):
            state: LilypadState = LilypadState(lilypad)
            d1: Dict[str,Categorical[Tuple[LilypadState, float]]] = {}

            # sound A
            sr_probs_dict_a: Dict[Tuple[LilypadState, float], float] = {}
            sr_probs_dict_a[(LilypadState(lilypad - 1), 0)] = lilypad / (self.num_lilypads)
            if lilypad + 1 == self.num_lilypads:
                sr_probs_dict_a[(LilypadState(lilypad + 1), 1)] = 1 - lilypad/(self.num_lilypads)
            else:
                sr_probs_dict_a[(LilypadState(lilypad + 1), 0)] = 1 - lilypad /(self.num_lilypads)
            d1['A'] = Categorical(sr_probs_dict_a)

            # sound B
            sr_probs_dict_b: Dict[Tuple[LilypadState, float], float] = {}
            for i in range(self.num_lilypads+1):
                if i != lilypad:
                    if i == self.num_lilypads :
                        sr_probs_dict_b[(LilypadState(i), 1)] = 1/(self.num_lilypads)
                    else:
                        sr_probs_dict_b[(LilypadState(i), 0)] = 1 /(self.num_lilypads)
            d1['B'] = Categorical(sr_probs_dict_b)

            d[state] = d1
        return d
    

In [15]:
from rl.dynamic_programming import value_iteration_result, policy_iteration_result

In [33]:
def get_all_val(number_of_lilypads: int, user_gamma=int):
    frog_mdp: FrogEscapeMDP = FrogEscapeMDP(number_of_lilypads)
    # CREATING ALL POSSIBLE DETERMINISTIC POLICIES
    policies = list(itertools.product(['A', 'B'], repeat=number_of_lilypads-1)) 
    val_function_map = {}
    for policy in policies:
        fdp: FinitePolicy[LilypadState, int] = FinitePolicy(
            {LilypadState(padnum):
            Constant(policy[padnum-1 ]) for padnum in range(1,number_of_lilypads)}
        )
        implied_mrp_tmp: FiniteMarkovRewardProcess[LilypadState] =\
            frog_mdp.apply_finite_policy(fdp)
        val_function_map[policy] = implied_mrp_tmp.get_value_function_vec(gamma=user_gamma)
    return val_function_map
def find_optimal(val_function_map):
    V_star = val_function_map[list(val_function_map.keys())[0]]
    for i, policy in enumerate(val_function_map):
        if np.all(val_function_map[policy] >= V_star) :
            V_star = val_function_map[policy] 
    pol_star = []        
    for policy in val_function_map.keys():
        if np.all(val_function_map[policy]  == V_star):
            pol_star = pol_star + [policy]
    return V_star, pol_star

In [34]:
time_summary = pd.DataFrame(index=range(2, 15), columns =['value_iteration','policy_iteration','brute_force'])

In [44]:
import time
import itertools
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
time_summary = pd.DataFrame(index=range(2, 20), columns =['value_iteration','policy_iteration','brute_force'])
for number_of_lilypads in range(2, 20):
    frog_mdp: FrogEscapeMDP = FrogEscapeMDP(number_of_lilypads)
    t1=time.time()
    opt_vf_vi, opt_policy_vi = value_iteration_result(frog_mdp, gamma=1)
    time_summary.loc[number_of_lilypads]['value_iteration'] = time.time()-t1
    t1=time.time()
    opt_vf_pi, opt_policy_pi = policy_iteration_result(frog_mdp, gamma=1)
    time_summary.loc[number_of_lilypads]['policy_iteration'] = time.time()-t1
    t1=time.time()
    brute_vi, brute_pi = find_optimal(get_all_val(number_of_lilypads=number_of_lilypads, user_gamma=1))
    time_summary.loc[number_of_lilypads]['brute_force'] = time.time()-t1

In [ ]:
time_summary.plot(title='time till convergence by different methods')
plt.xlabel('# of lilypads');

Value iteration is the fastest to converge.

# Question 3: Job-Hopping and Wages-Utility-Maximization

Setup:

- Start of the Day: {E, U}

- During the day

    - If employed, earn wage $w_n$; 
    
    - If unemployed, randomly offered one of n jobs with respective probability. Choose accept/decline. If accept, earn $w_n$, else get unemployment wage $w_0$.
    
- End of day, lose job with prob $\alpha \in [0,1]$

Objective: maximize infinite horizon expected discounted sum of wage utility. $max \mathbb{E} [\sum_{u=t}^\infty \gamma^{u-t} \cdot log(w_i)]$

$\gamma \in [0, 1$, $U(w) = log(w)$

## Part 1: Notations

## Part 2: Solve in python

# Question 4: 2-stores inventory control